In [78]:
import logging
try:
    from Queue import Queue  # PY2
except ImportError:
    from queue import Queue  # PY3
from threading import Thread
try:
    from urlparse import urljoin  # PY2
except ImportError:
    from urllib.parse import urljoin  # PY3

from bs4 import BeautifulSoup
import requests
from requests.exceptions import RequestException
from six import iteritems
from six.moves import range

sites_url = 'http://www.craigslist.org/about/sites'


def get_all_sites():
    response = requests.get(sites_url)
    response.raise_for_status()  # Something failed?
    soup = BeautifulSoup(response.content, 'html.parser')
    sites = set()

    for box in soup.findAll('div', {'class': 'box'}):
        for a in box.findAll('a'):
            # Remove protocol and get subdomain
            site = a.attrs['href'].rsplit('//', 1)[1].split('.')[0]
            sites.add(site)

    return sites
ALL_SITES = get_all_sites()  # All the Craiglist sites
RESULTS_PER_REQUEST = 100  # Craigslist returns 100 results per request

In [79]:
def requests_get(*args, **kwargs):
    """
    Retries if a RequestException is raised (could be a connection error or
    a timeout).
    """

    logger = kwargs.pop('logger', None)
    try:
        return requests.get(*args, **kwargs)
    except RequestException as exc:
        if logger:
            logger.warning('Request failed (%s). Retrying ...', exc)
        return requests.get(*args, **kwargs)

In [80]:
def get_list_filters(url):
    list_filters = {}
    response = requests_get(url)
    soup = BeautifulSoup(response.content, 'html.parser')
    for list_filter in soup.find_all('div', class_='search-attribute'):
        filter_key = list_filter.attrs['data-attr']
        filter_labels = list_filter.find_all('label')
        options = [opt.text.strip() for opt in filter_labels]
        list_filters[filter_key] = {'url_key': filter_key, 'value': options}
    return list_filters

In [176]:
from requests_futures.sessions import FuturesSession
import re
class CraigslistBase(object):
    """ Base class for all Craiglist wrappers. """

    url_templates = {
        'base': 'http://%(site)s.craigslist.org',
        'no_area': 'http://%(site)s.craigslist.org/search/%(category)s',
        'area': 'http://%(site)s.craigslist.org/search/%(area)s/%(category)s'
    }

    default_site = 'sfbay'
    default_category = None

    base_filters = {
        'query': {'url_key': 'query', 'value': None},
        'search_titles': {'url_key': 'srchType', 'value': 'T'},
        'has_image': {'url_key': 'hasPic', 'value': 1},
        'posted_today': {'url_key': 'postedToday', 'value': 1},
        'search_distance': {'url_key': 'search_distance', 'value': None},
        'zip_code': {'url_key': 'postal', 'value': None},
    }
    extra_filters = {}

    # Set to True to subclass defines the customize_results() method
    custom_result_fields = False

    sort_by_options = {
        'newest': 'date',
        'price_asc': 'priceasc',
        'price_desc': 'pricedsc',
    }

    def __init__(self, site=None, area=None, category=None, filters=None,
                 log_level=logging.WARNING):
        # Logging
        self.set_logger(log_level, init=True)

        self.site = site or self.default_site
        if self.site not in ALL_SITES:
            msg = "'%s' is not a valid site" % self.site
            self.logger.error(msg)
            raise ValueError(msg)

        if area:
            if not self.is_valid_area(area):
                msg = "'%s' is not a valid area for site '%s'" % (area, site)
                self.logger.error(msg)
                raise ValueError(msg)
        self.area = area

        self.category = category or self.default_category

        url_template = self.url_templates['area' if area else 'no_area']
        self.url = url_template % {'site': self.site, 'area': self.area,
                                   'category': self.category}

        list_filters = get_list_filters(self.url)

        self.filters = {}
        for key, value in iteritems((filters or {})):
            try:
                filter = (self.base_filters.get(key) or
                          self.extra_filters.get(key) or
                          list_filters[key])
                if filter['value'] is None:
                    self.filters[filter['url_key']] = value
                elif isinstance(filter['value'], list):
                    valid_options = filter['value']
                    if not hasattr(value, '__iter__'):
                        value = [value]  # Force to list
                    options = []
                    for opt in value:
                        try:
                            options.append(valid_options.index(opt) + 1)
                        except ValueError:
                            self.logger.warning(
                                "'%s' is not a valid option for %s"
                                % (opt, key)
                            )
                    self.filters[filter['url_key']] = options
                elif value:  # Don't add filter if ...=False
                    self.filters[filter['url_key']] = filter['value']
            except KeyError:
                self.logger.warning("'%s' is not a valid filter", key)

    def set_logger(self, log_level, init=False):
        if init:
            self.logger = logging.getLogger('python-craiglist')
            self.handler = logging.StreamHandler()
            self.logger.addHandler(self.handler)
        self.logger.setLevel(log_level)
        self.handler.setLevel(log_level)

    def is_valid_area(self, area):
        base_url = self.url_templates['base']
        response = requests_get(base_url % {'site': self.site},
                                logger=self.logger)
        soup = BeautifulSoup(response.content, 'html.parser')
        sublinks = soup.find('ul', {'class': 'sublinks'})
        return sublinks and sublinks.find('a', text=area) is not None

    def get_results(self, limit=None, start=0, sort_by=None, geotagged=False):
        """
        Get results from Craigslist based on the specified filters.
        If geotagged=True, the results will include the (lat, lng) in the
        'geotag' attrib (this will make the process a little bit longer).
        """

        if sort_by:
            try:
                self.filters['sort'] = self.sort_by_options[sort_by]
            except KeyError:
                msg = ("'%s' is not a valid sort_by option, "
                       "use: 'newest', 'price_asc' or 'price_desc'" % sort_by)
                self.logger.error(msg)
                raise ValueError(msg)

        total_so_far = start
        results_yielded = 0
        total = 0

        while True:
            self.filters['s'] = start
            response = requests_get(self.url, params=self.filters,
                                    logger=self.logger)
            self.logger.info('GET %s', response.url)
            self.logger.info('Response code: %s', response.status_code)
            response.raise_for_status()  # Something failed?

            soup = BeautifulSoup(response.content, 'html.parser')
            if not total:
                totalcount = soup.find('span', {'class': 'totalcount'})
                total = int(totalcount.text) if totalcount else 0

            for row in soup.find_all('p', {'class': 'result-info'}):
                if limit is not None and results_yielded >= limit:
                    break
                self.logger.debug('Processing %s of %s results ...',
                                  total_so_far + 1, total)

                link = row.find('a', {'class': 'hdrlnk'})
                id = link.attrs['data-id']
                name = link.text
                url = urljoin(self.url, link.attrs['href'])

                time = row.find('time')
                if time:
                    datetime = time.attrs['datetime']
                else:
                    pl = roprintw.find('span', {'class': 'pl'})
                    datetime = pl.text.split(':')[0].strip() if pl else None
                price = row.find('span', {'class': 'result-price'})
                where = row.find('span', {'class': 'result-hood'})
                if where:
                    where = where.text.strip()[1:-1]  # remove ()
                tags_span = row.find('span', {'class': 'result-tags'})
                tags = tags_span.text if tags_span else ''

                result = {#'id': id,
                          'name': name,
                          'url': url,
                          'datetime': datetime,
                          'price': price.text if price else None,
                          'where': where,
                          'has_image': 'pic' in tags#,
                          # TODO: Look into this, looks like all show map now
                          #'has_map': 'map' in tags,
                #          'geotag': None
                }
                
                session=FuturesSession()
                future = session.get(url)
                response_detail= future.result()
                soup_response_detail=BeautifulSoup(response_detail.text,"lxml")
                
                try:
                    mapaddress_pre=soup_response_detail.find_all(['div','p'], {'class': 'mapaddress'})
                    mapaddress=''
                    for i in mapaddress_pre:
                        mapaddress=mapaddress+i.text.replace('\n\n','\n')
                    #mapaddress=mapaddress_pre[0].text.replace('\n\n','\n')
                    result.update({'mapaddress':mapaddress})
                except:
                    pass
                
                try:
                    detail_content_pre=soup_response_detail.find_all(id='postingbody')
                    detail_content=''
                    for i in detail_content_pre:
                        detail_content=detail_content+i.text.replace(unicode('\n\nQR Code Link to This Post\n\n\n'),'')\
                        .replace('\n\n','\n')

                    #detail_content=detail_content_pre[0].text.replace(unicode('\n\nQR Code Link to This Post\n\n\n'),'')\
                    #.replace('\n\n','\n')
                    result.update({'detail_content':detail_content})


                    chinese_flag=re.findall(ur'[\u4e00-\u9fff]+', detail_content)


                    if len(chinese_flag)>0:
                        chinese_content=True
                    else:
                        chinese_content=False
                    result.update({'chinese_content':chinese_content})
                except:
                    pass
                
                
                
                try:
                    attrgroup_pre=soup_response_detail.find_all(['div','p'], {'class': 'attrgroup'})
                    attr_text=''
                    for tmp in attrgroup_pre:
                        attr_text=attr_text+tmp.text.replace('\n\n','\n')
                    result.update({'attr_text':attr_text})
                except:
                    pass
                
                
                geolocation=geolocation=soup_response_detail.find_all(['div','p'], {'class': 'viewposting'})
                try:
                    geolocation=geolocation[0]
                    geolocation_latitude=geolocation.attrs['data-latitude']
                    geolocation_longitude=geolocation.attrs['data-longitude']
                    result.update({'geolocation_latitude':geolocation_latitude})
                    result.update({'geolocation_longitude':geolocation_longitude})
                except:
                    pass
                
                
                if self.custom_result_fields:
                    self.customize_result(result, row)

                if geotagged and result['has_map']:
                    self.geotag_result(result)

                yield result
                results_yielded += 1
                total_so_far += 1

            if results_yielded == limit:
                break
            if (total_so_far - start) < RESULTS_PER_REQUEST:
                break
            start = total_so_far

    def customize_result(self, result, html_row):
        """ Add custom/delete/alter fields to result. """
        pass  # Override in subclass to add category-specific fields.

    def geotag_result(self, result):
        """ Adds (lat, lng) to result. """

        self.logger.debug('Geotagging result ...')

        if result['has_map']:
            response = requests_get(result['url'], logger=self.logger)
            self.logger.info('GET %s', response.url)
            self.logger.info('Response code: %s', response.status_code)

            if response.ok:
                soup = BeautifulSoup(response.content, 'html.parser')
                map = soup.find('div', {'id': 'map'})
                if map:
                    result['geotag'] = (float(map.attrs['data-latitude']),
                                        float(map.attrs['data-longitude']))

        return result

    def geotag_results(self, results, workers=8):
        """
        Add (lat, lng) to each result. This process is done using N threads,
        where N is the amount of workers defined (default: 8).
        """

        results = list(results)
        queue = Queue()

        for result in results:
            queue.put(result)

        def geotagger():
            while not queue.empty():
                self.logger.debug('%s results left to geotag ...',
                                  queue.qsize())
                self.geotag_result(queue.get())
                queue.task_done()

        threads = []
        for _ in range(workers):
            thread = Thread(target=geotagger)
            thread.start()
            threads.append(thread)

        for thread in threads:
            thread.join()
        return results

    @classmethod
    def show_filters(cls, category=None):
        print('Base filters:')
        for key, options in iteritems(cls.base_filters):
            value_as_str = '...' if options['value'] is None else 'True/False'
            print('* %s = %s' % (key, value_as_str))
        print('Section specific filters:')
        for key, options in iteritems(cls.extra_filters):
            value_as_str = '...' if options['value'] is None else 'True/False'
            print('* %s = %s' % (key, value_as_str))
        url = cls.url_templates['no_area'] % {
            'site': cls.default_site,
            'category': category or cls.default_category,
        }
        list_filters = get_list_filters(url)
        for key, options in iteritems(list_filters):
            value_as_str = ', '.join([repr(opt) for opt in options['value']])
            print('* %s = %s' % (key, value_as_str))

            

In [177]:
class CraigslistHousing(CraigslistBase):
    """ Craigslist housing wrapper. """

    default_category = 'hhh'
    custom_result_fields = True

    extra_filters = {
        'private_room': {'url_key': 'private_room', 'value': 1},
        'private_bath': {'url_key': 'private_bath', 'value': 1},
        'cats_ok': {'url_key': 'pets_cat', 'value': 1},
        'dogs_ok': {'url_key': 'pets_dog', 'value': 1},
        'min_price': {'url_key': 'min_price', 'value': None},
        'max_price': {'url_key': 'max_price', 'value': None},
        'min_ft2': {'url_key': 'minSqft', 'value': None},
        'max_ft2': {'url_key': 'maxSqft', 'value': None},
        'min_bedrooms': {'url_key': 'min_bedrooms', 'value': None},
        'max_bedrooms': {'url_key': 'max_bedrooms', 'value': None},
        'min_bathrooms': {'url_key': 'min_bathrooms', 'value': None},
        'max_bathrooms': {'url_key': 'max_bathrooms', 'value': None},
        'no_smoking': {'url_key': 'no_smoking', 'value': 1},
        'is_furnished': {'url_key': 'is_furnished', 'value': 1},
        'wheelchair_acccess': {'url_key': 'wheelchaccess', 'value': 1},
    }

    def customize_result(self, result, html_row):
        housing_info = html_row.find('span', {'class': 'housing'})
        # Default values
        result.update({'bedrooms': None, 'area': None})
        if housing_info:
            for elem in housing_info.text.split('-'):
                elem = elem.strip()
                if elem.endswith('br'):
                    # Don't convert to int, too risky
                    result['bedrooms'] = elem[:-2]
                if elem.endswith('2'):
                    result['area'] = elem

In [172]:
cl_h = CraigslistHousing(site='sfbay', area='sby', category='roo',
                        # filters={'max_price': 2000, 'private_room': True}
                        )

In [178]:
apa=CraigslistHousing(site='sfbay', area='sby', category='apa',
                     #    filters={'max_price': 2000, 'private_room': True}
                     )

In [194]:
import pandas as pd
#pd.set_option('display.max_colwidth', -1)
pd.set_option('display.max_colwidth',3)
df = pd.DataFrame(cl_h.get_results(sort_by='newest', limit=1))
df

,area,attr_text,bedrooms,chinese_content,datetime,detail_content,geolocation_latitude,geolocation_longitude,has_image,mapaddress,name,price,url,where
0,150ft2,\n150ft2\navailable may 1\n\nw/d in unit\nno smoking\ncarport\nprivate bath\nprivate room\n,None,False,2017-07-03 15:01,"Nice, Large, Bright, room W/ Semi private Bath, Private entrance, Security, and Covered driveway parking Available for Rent.Room is 13 x 11.5 feet. Furnished or unfurnished. Amenities include: Swimming pool, Jacuzzi, Weight rooms, Sauna, Tennis court, Billiards room, Library, and Clubhouse. Wifi, cable, and all utilities included. \nCentrally located for easy commute.\nEasy access to Hwy 85/87/280/880/101 Etc.\nMinutes away from local shopping & restaurants.\nClose to Monterey Rd. and Tully\Curtner (Fairgrounds)\nWalking distance to VTA bus stop, Light Rail, and Cal Train, Shopping, and Restaurants. Quiet & safe neighborhood.\n",37.294437,-121.852756,True,\n (google map)\n \n,"NICE, LARGE ROOM W/ SEMI PRIVATE BATH, PRIVATE ENTRANCE, SECURITY, AND",$1150,http://sfbay.craigslist.org/sby/roo/6179828056.html,san jose south


In [196]:
import pandas as pd
pd.set_option('display.max_colwidth', -1)
df = pd.DataFrame(apa.get_results(sort_by='newest'))
df.head(2)

ConnectionError: HTTPConnectionPool(host='sfbay.craigslist.org', port=80): Max retries exceeded with url: /sby/apa/6203234781.html (Caused by NewConnectionError('<requests.packages.urllib3.connection.HTTPConnection object at 0x000000000D7F18D0>: Failed to establish a new connection: [Errno 10060] A connection attempt failed because the connected party did not properly respond after a period of time, or established connection failed because connected host has failed to respond',))